### Basic Environment Set Up and Check

In [20]:
# Check if .env Exists
import os
print(".env file exists:", os.path.exists(".env"))

.env file exists: True


In [21]:
print(os.listdir())  # Lists all files in the current directory

['.DS_Store', 'bin', 'include', 'etc', 'pyvenv.cfg', '.env', 'lib', '.ipynb_checkpoints', 'github_trend_analysis.ipynb', 'share']


In [22]:
import os 
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the GitHub Token
github_token = os.getenv("GITHUB_TOKEN")

# Test if the token is loaded
if github_token:
    print("GitHub Token loaded successfully!")
else:
    print("Failed to load GitHub Token. Check your .env file.")

GitHub Token loaded successfully!


In [24]:
import requests

headers = {
    "Authorization": f"token {os.environ['GITHUB_TOKEN']}"
}

# Test GitHub API
url = "https://api.github.com/user"
response = requests.get(url, headers=headers)

# Check if the API implemented successfully
if response.status_code == 200:
    print("GitHub API Token Set Successfully！")
    print(response.json())  # Show the user indfo
else:
    print(f"Error: {response.status_code}, {response.text}")

GitHub API Token Set Successfully！
{'login': 'sydneyci11', 'id': 114530581, 'node_id': 'U_kgDOBtOZFQ', 'avatar_url': 'https://avatars.githubusercontent.com/u/114530581?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/sydneyci11', 'html_url': 'https://github.com/sydneyci11', 'followers_url': 'https://api.github.com/users/sydneyci11/followers', 'following_url': 'https://api.github.com/users/sydneyci11/following{/other_user}', 'gists_url': 'https://api.github.com/users/sydneyci11/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/sydneyci11/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/sydneyci11/subscriptions', 'organizations_url': 'https://api.github.com/users/sydneyci11/orgs', 'repos_url': 'https://api.github.com/users/sydneyci11/repos', 'events_url': 'https://api.github.com/users/sydneyci11/events{/privacy}', 'received_events_url': 'https://api.github.com/users/sydneyci11/received_events', 'type': 'User', 'user_view_type': 'privat

### Fetch Repository Data from GitHub API and Store to Database

In [19]:
import requests
import sqlite3
import pandas as pd
from dotenv import load_dotenv
import os

# Define the new database path (local storage)
local_db_path = "/Users/sydneyci11/Documents/github-trend-analysis/github_trends.db"

# Ensure the directory exists
os.makedirs(os.path.dirname(local_db_path), exist_ok=True)

# Create and connect to the database
conn = sqlite3.connect(local_db_path)
cursor = conn.cursor()

# Create the table (if not exists)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS github_trends (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        full_name TEXT,
        description TEXT,
        stars INTEGER,
        forks INTEGER,
        language TEXT,
        created_at TEXT,
        updated_at TEXT,
        url TEXT
    )
""")
conn.commit()
conn.close()  # Close connection after creating table

print(f"Database successfully created at: {local_db_path}")

# Load API token from .env
load_dotenv()
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

# Define GitHub search query
query = "machine learning"
url = f"https://api.github.com/search/repositories?q={query}&sort=stars&order=desc"

# Fetch data from GitHub API
response = requests.get(url, headers=headers)
data = response.json()

# Check response
if response.status_code == 200:
    print(f"Successfully fetched {len(data['items'])} repositories!")
else:
    print(f"Error: {response.status_code}, {response.text}")

# Function to insert data into the local database
def save_to_db(repo_data):
    # Reopen database connection inside function
    conn = sqlite3.connect(local_db_path)
    cursor = conn.cursor()
    
    query = """INSERT INTO github_trends (name, full_name, description, stars, forks, language, created_at, updated_at, url) 
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    
    cursor.execute(query, repo_data)
    conn.commit()
    conn.close()  # Close connection after inserting

# Store data into the local database
for repo in data["items"]:
    repo_data = (
        repo["name"], repo["full_name"], repo["description"],
        repo["stargazers_count"], repo["forks_count"], repo["language"],
        repo["created_at"], repo["updated_at"], repo["html_url"]
    )
    save_to_db(repo_data)

print(f"Data successfully saved to: {local_db_path}")


Database successfully created at: /Users/sydneyci11/Documents/github-trend-analysis/github_trends.db
Successfully fetched 30 repositories!
Data successfully saved to: /Users/sydneyci11/Documents/github-trend-analysis/github_trends.db


### Inspect Data

In [27]:
import sqlite3
import pandas as pd

# Connect to the local database
conn = sqlite3.connect("/Users/sydneyci11/Documents/github-trend-analysis/github_trends.db")

# Read some data
df = pd.read_sql("SELECT * FROM github_trends LIMIT 10;", conn)
conn.close()

df  # Display the first few rows

,id,name,full_name,description,stars,forks,language,created_at,updated_at,url
0,1,tensorflow,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,188713,74612,C++,2015-11-07T01:19:20Z,2025-03-18T19:18:01Z,https://github.com/tensorflow/tensorflow
1,2,transformers,huggingface/transformers,🤗 Transformers: State-of-the-art Machine Learn...,141439,28326,Python,2018-10-29T13:56:00Z,2025-03-18T18:54:42Z,https://github.com/huggingface/transformers
2,3,funNLP,fighting41love/funNLP,中英文敏感词、语言检测、中外手机/电话归属地/运营商查询、名字推断性别、手机号抽取、身份证抽...,71747,14747,Python,2018-08-21T11:20:39Z,2025-03-18T17:46:05Z,https://github.com/fighting41love/funNLP
3,4,ML-For-Beginners,microsoft/ML-For-Beginners,"12 weeks, 26 lessons, 52 quizzes, classic Mach...",71470,15281,HTML,2021-03-03T01:34:05Z,2025-03-18T18:13:12Z,https://github.com/microsoft/ML-For-Beginners
4,5,awesome-machine-learning,josephmisiti/awesome-machine-learning,A curated list of awesome Machine Learning fra...,67243,14812,Python,2014-07-15T19:11:19Z,2025-03-18T18:07:54Z,https://github.com/josephmisiti/awesome-machin...
5,6,scikit-learn,scikit-learn/scikit-learn,scikit-learn: machine learning in Python,61453,25690,Python,2010-08-17T09:43:38Z,2025-03-18T19:10:41Z,https://github.com/scikit-learn/scikit-learn
6,7,gradio,gradio-app/gradio,Build and share delightful machine learning ap...,36906,2802,Python,2018-12-19T08:24:04Z,2025-03-18T18:55:24Z,https://github.com/gradio-app/gradio
7,8,C-Plus-Plus,TheAlgorithms/C-Plus-Plus,Collection of various algorithms in mathematic...,31448,7330,C++,2016-07-16T10:19:45Z,2025-03-18T17:44:09Z,https://github.com/TheAlgorithms/C-Plus-Plus
8,9,netron,lutzroeder/netron,"Visualizer for neural network, deep learning a...",29666,2863,JavaScript,2010-12-26T12:53:43Z,2025-03-18T13:42:20Z,https://github.com/lutzroeder/netron
9,10,linkedin-skill-assessments-quizzes,Ebazhanov/linkedin-skill-assessments-quizzes,Full reference of LinkedIn answers 2024 for sk...,28601,13251,None,2019-07-13T19:52:56Z,2025-03-18T15:48:44Z,https://github.com/Ebazhanov/linkedin-skill-as...
